Ignis: Latent Dirichlet Allocation
============

In [1]:
import ignis

In [2]:
import ipywidgets as widgets

Model training (LDA)
----

Load from an `ignis.Corpus`, add the processed docs to an LDA model, and train it.

The random seed and parallelisation can both affect results, so setting the seed and number of workers is necessary for reproducibility.

In [3]:
corpus = ignis.load_corpus("bbc-full.corpus")

In [4]:
model_options = {
    "k": 10,
    "term_weighting": "idf",
    "until_max_ll": True,
    "verbose": True,
}
vis_options = {"verbose": True}
results = ignis.train_model(
    corpus,
    model_type="tp_lda",
    model_options=model_options,
    vis_type="pyldavis",
    vis_options=vis_options,
)

Training LDA model on 2225 documents:
{'term_weighting': 'idf', 'k': 10, 'seed': 11399, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:12<00:00, 82.52it/s, Log-likelihood=-20.93906]



Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)



3300it [00:47, 69.29it/s, Log-likelihood=-20.68781]


Model training complete. (60.788s)
Preparing LDA visualisation . . . . . . . . . . . . . . Done. (17.710s)


In [5]:
results.save("bbc-full.aurum")
results = ignis.load_results("bbc-full.aurum")

In [6]:
results.init_labeller("tomotopy", verbose=True)

Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.


Graphical visualisation
--------

In [7]:
results.show_visualisation()

In [8]:
results.export_visualisation("visualisation")

Exploring the top words and suggested labels for each topic
------

In [9]:
results.nb_show_topics(top_labels=15, top_words=15)

interactive(children=(IntSlider(value=1, description='topic_id', max=10, min=1), Output()), _dom_classes=('wid…

<function ignis.aurum.Aurum.nb_show_topics.<locals>.show_topic(topic_id=1)>

## Exploring documents that "belong" to a given topic

Because topics are distributions over words and documents are *distributions* over topics, documents don't belong to individual topics per se; every topic is represented in every document with some probability.

We therefore have to specify how many of the document's top `n` topics we want to check for the actual topic we're interested in.

This is especially significant for topic models that use a term weighting scheme, because all the common words (i.e., what we might consider stopwords) tend to get grouped into a single large topic; if we only consider each document's single most probable topic, we will unintentionally exclude documents which have this "stopwords" topic as their top topic.

In [10]:
# Topic 4 appears to be related to the entertainment industry
results.nb_show_topic_documents(topic_id=4, within_top_n=2)

interactive(children=(IntSlider(value=0, description='index', max=369), Output()), _dom_classes=('widget-inter…

<function ignis.aurum.Aurum.nb_show_topic_documents.<locals>.show_topic_doc(index=0)>

Slicing and iteration
--------
After seeing what the main topics might be, we can slice the initial corpus further and re-run topic modelling to get better resolution.

In [11]:
# Try zooming in on Topic 4
entertainment_slice = results.slice_by_topic(topic_id=4, within_top_n=2)

entertainment_model = results.retrain_model(corpus_slice=entertainment_slice)

Training LDA model on 370 documents:
{'term_weighting': 'idf', 'k': 10, 'seed': 11399, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': True, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:02<00:00, 339.79it/s, Log-likelihood=-18.74928]


Continuing to train until maximum log-likelihood.
(N.B.: This may not correlate with increased interpretability)




700it [00:02, 272.80it/s, Log-likelihood=-18.70620]

Model training complete. (5.731s)
Extracting label candidates from model...


Preparing First-order relevance labeller...
Done.
Preparing LDA visualisation . . . . . . Done. (7.233s)


In [12]:
entertainment_model.show_visualisation()

The position of each topic cluster on the graph is not intrinsically informative per se (being simply the result of some specified dimensionality-reducing technique), but if we want we can run the modelling algorithm with a different random seed and see if we get a more meaningful set of topics.

(We can also override any of the previously set options.)

In [13]:
model_options = {"seed": 1234, "until_max_ll": False}
entertainment_model_2 = entertainment_model.retrain_model(model_options=model_options)
entertainment_model_2.show_visualisation()

Training LDA model on 370 documents:
{'term_weighting': 'idf', 'k': 10, 'seed': 1234, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 1000, 'update_every': 100, 'until_max_ll': False, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 1000/1000 [00:02<00:00, 334.00it/s, Log-likelihood=-18.80618]

Model training complete. (2.997s)
Extracting label candidates from model...
Preparing First-order relevance labeller...
Done.
Preparing LDA visualisation

 . . . . . Done. (6.868s)


Alternatively, we can slice the corpus by explicitly searching for documents that contain certain tokens, in case we want to be absolutely sure we got all the documents that mention certain words or phrases.

In [14]:
# Topics that seem to deal with films
film_1 = entertainment_model_2.slice_by_topic(topic_id=10, within_top_n=2)
film_2 = entertainment_model_2.slice_by_topic(topic_id=3, within_top_n=2)

# Tokens that are related to films
by_token_slice = entertainment_model_2.slice_by_tokens(
    ["film", "films", "movie", "cinema"], include_root=True
)

concat_slice = film_1.concat(film_2, by_token_slice)

model_options = {"iterations": 2000, "k": 5}
concat_model = entertainment_model_2.retrain_model(
    concat_slice, model_options=model_options
)
concat_model.show_visualisation()

Training LDA model on 322 documents:
{'term_weighting': 'idf', 'k': 5, 'seed': 1234, 'workers': 8, 'parallel_scheme': 'default', 'iterations': 2000, 'update_every': 100, 'until_max_ll': False, 'max_extra_iterations': 5000, 'verbose': True, 'tw': <TermWeight.IDF: 1>, 'parallel': <ParallelScheme.DEFAULT: 0>}



100%|██████████| 2000/2000 [00:05<00:00, 376.96it/s, Log-likelihood=-18.09508]

Model training complete. (5.309s)
Extracting label candidates from model...
Preparing First-order relevance labeller...


Done.
Preparing LDA visualisation . . . . . Done. (6.980s)


In [16]:
# Topic 3 appears to be related to film awards
concat_model.nb_show_topic_documents(topic_id=3, within_top_n=2)

interactive(children=(IntSlider(value=0, description='index', max=156), Output()), _dom_classes=('widget-inter…

<function ignis.aurum.Aurum.nb_show_topic_documents.<locals>.show_topic_doc(index=0)>